# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification, unify
from copy import deepcopy

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

## parse_state

`parse_state` is a helper function that parses an input state in `forward_planner`. It creates one string from the state parameter, parses the string, and returns a set of tuples with each tuple as a predicate in the state. **Used by**: [parse_actions](#parse_actions), [forward_planner](#forward_planner).

* **state**: the input state to parse as a list of strings of predicates

**returns** `Set(Tuple)`: a set of tuples representing the state

In [2]:
def parse_state(state):
    res = '(' + ''.join(state) + ')'
    new_res = parse(res)
    return set(tuple(x) for x in new_res)

In [3]:
# assertions/unit tests
start_state = [ "(item Saw)", "(item Drill)", "(place Home)",
               "(place Store)", "(place Bank)",  "(agent Me)",
               "(at Me Home)", "(at Saw Store)", "(at Drill Store)"]

parsed_start = parse_state(start_state)
assert ("item", "Saw") in parsed_start
assert ("at", "Me", "Home") in parsed_start
assert ("at", "Me", "Store") not in parsed_start

## parse_actions

`parse_actions` takes a list of actions and parses each action by transforming the action, add list, and delete lists to state representation (a set of tuples with each tuple as a condition/action. **Uses**: [parse_state](#parse_state). **Used by**: [forward_planner](#forward_planner).

* **actions**: a dictionary of actions, with each action having an action, conditions, add, and delete lists

**returns** `Dict`: a new dictionary with parsed values for each action

In [4]:
def parse_actions(actions):
    new_actions = deepcopy(actions)
    for action in new_actions:
        new_actions[action]["conditions"] = parse_state(new_actions[action]["conditions"])
        new_actions[action]["add"] = parse_state(new_actions[action]["add"])
        new_actions[action]["delete"] = parse_state(new_actions[action]["delete"])
        new_actions[action]["action"] = parse_state(new_actions[action]["action"])
    return new_actions

In [5]:
# assertions/unit tests
actions = {
    "fly": {
        "action": "(fly ?plane ?to ?from)",
        "conditions": [
            "(plane ?plane)"
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)"
        ],
        "add": ["(at ?plane ?to)"],
        "delete": ["(at ?plane ?from)"]
    },
}

parsed_actions = parse_actions(actions)
assert parsed_actions['fly']['action'] == {('fly', '?plane', '?to', '?from')}
assert parsed_actions['fly']['conditions'] == {('at', '?plane', '?from'), ('airport', '?to'), ('airport', '?from'), ('plane', '?plane')}
assert parsed_actions['fly']['add'] == {('at', '?plane', '?to')}
assert parsed_actions['fly']['delete'] == {('at', '?plane', '?from')}

## apply_subs

`apply_subs` takes a substitution list of variables and a state set and applies variable substitutions to the state set. This function is used to transform possible actions after a substitution list is found (once the precondition set unifies with the current state). The function returns a new state with substitutions applied. **Used by**: [successors](#successors).

* **sub_list**: a dictionary of substitutions
* **state**: a set of tuples with variables to be substituted

**returns** `Set(Tuple)`: a new set of tuples with substitutions applied

In [6]:
def apply_subs(sub_list, state):
    new_state = set()
    for predicate in state:
        new_pred = [sub_list[x] if x in sub_list else x for x in predicate]
        new_state.add(tuple(new_pred))
    return new_state

In [7]:
sub_list = {'?plane': '1973'}
state = {('plane', '?plane')}
res = apply_subs(sub_list, state)
assert res == {('plane', '1973')}

state = {('at', '?plane', '?from'), ('plane', '?plane'), ('airport', '?to'), ('airport', '?from')}
res = apply_subs(sub_list, state)
assert res == {('at', '1973', '?from'), ('airport', '?to'), ('plane', '1973'), ('airport', '?from')}

state = {('at', '?plane', '?from'), ('plane', '?plane'), ('airport', '?to'), ('airport', '?from')}
sub_list = {'?plane': '1973', '?to': 'JFK', '?from': 'SFO'}
res = apply_subs(sub_list, state)
assert res == {('airport', 'SFO'), ('at', '1973', 'SFO'), ('plane', '1973'), ('airport', 'JFK')}

## precon_search

`precon_search` determines if a state unifies with the conditions list of a possible action. To do so, it performs a linear search with recursive backtracking. It compares each condition in `conditions` to each predicate in `start_state` - if the two facts unify, the resulting substitution is added to a substitution list and the two facts are removed from their lists before the recursive call. 

Recursive backtracking allows the function to find all possible ways a set of conditions can unify with a state - the total list of substitution lists is stored in the parameter `all_subs` and is mutated to include all the substitutions by the end of the function. The two return parameters, `subs` and `success` are used purely for execution and have no bearing on the overall success of the search. **Uses**: [unification](#unification) **Used by**: [successors](#successors).

* **conditions**: a list of conditions for an action
* **start_state**: the current state being checked against the conditions
* **subs**: a dictionary of current substitutions that is mutated on each recursive call
* **all_subs**: a list of substitution lists that contains the result of the search

**returns** `Tuple(List, Bool)`: the current substitution list and a success parameter, used internally for the algorithm (they should not be used for checking the success/results of `precon_search` elsewhere).

In [8]:
def precon_search(conditions, start_state, subs, all_subs):
    cons, preds = deepcopy(conditions), deepcopy(start_state)
    if len(cons) == 0: return subs, True
    for con in cons:
        for pred in preds:
            check_uni = unification(con, pred)
            if check_uni != False:
                new_cons = deepcopy(apply_subs(check_uni, [x for x in cons if x != con]))
                new_preds = deepcopy([y for y in preds if y != pred])
                subs.update(check_uni);
                subs, success = precon_search(new_cons, new_preds, deepcopy(subs), all_subs)
                if success: 
                    if subs not in all_subs: all_subs.append(subs)
            if type(check_uni) == dict: 
                subs = {k: v for k,v in subs.items() if k not in check_uni}
    return subs, False

In [9]:
# assertions/unit tests
actions = {
    "fly": {
        "action": "(fly ?plane ?to ?from)",
        "conditions": [
            "(plane ?plane)"
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)"
        ],
        "add": ["(at ?plane ?to)"],
        "delete": ["(at ?plane ?from)"]
    },
}
p_actions = parse_actions(actions)
actions_compare = deepcopy(p_actions)

cons = p_actions["fly"]["conditions"]
start_state = ["(plane 1973)", "(airport SFO)", "(airport JFK)", "(at 1973 SFO)"]
p_state = parse_state(start_state)
all_subs = []
res, success = precon_search(cons, p_state, {}, all_subs)
assert all_subs == [{'?plane': '1973', '?from': 'SFO', '?to': 'JFK'}]

start_state = ["(plane 1973)", "(plane 2749)", "(airport SFO)",
               '(airport JFK)', "(airport ORD)", "(at 1973 SFO)",
               "(at 2749 JFK)", "(at 97 ORD)","(at 1211 SFO)"]
p_state = parse_state(start_state)
state_compare = deepcopy(p_state)
all_subs = []
res, success = precon_search(cons, p_state, {}, all_subs)
actual = [{'?plane': '2749', '?from': 'JFK', '?to': 'ORD'}, 
                    {'?plane': '2749', '?from': 'JFK', '?to': 'SFO'}, 
                    {'?plane': '1973', '?from': 'SFO', '?to': 'ORD'}, 
                    {'?plane': '1973', '?from': 'SFO', '?to': 'JFK'}]
assert all(x in actual for x in all_subs)
assert p_actions == actions_compare and p_state == state_compare

## successors

`successors` is the main method of deepening in our `forward_planner` algorithm. This function takes a parent node, which has a state, action, and its own parent, and tries to apply every action in `actions`. If the action unifies from `precon_search`, then every substitution list in the `all_subs` parameter mutated by `precon_search` is used to generate a new child. The add/delete actions are taken and the child is added to a list of children.

This function is vital to the overall forward planning algorithm - the process of unifying preconditions and creating children constitutes a secondary state space search inside the main search of `forward_planner`. **Uses**: [precon_search](#precon_search), [apply_subs](#apply_subs). **Used by**: [forward_planner](#forward_planner).

* **parent**: a parent state from which to generate children
* **actions**: a list of possible actions

**returns** `List`: a list of child states from `parent` through `actions`.

In [10]:
def successors(parent, actions):
    (state, action, parent_state), children = parent, []
    for action in actions:
        all_subs = []
        _, _ = precon_search(actions[action]["conditions"], state, {}, all_subs)
        for sub_list in all_subs:
            add_sub = apply_subs(sub_list, actions[action]["add"])
            del_sub = apply_subs(sub_list, actions[action]["delete"])
            action_sub = list(apply_subs(sub_list, actions[action]["action"]))[0]
            child_state = [con for con in add_sub]
            for con in state: 
                if con not in del_sub: child_state.append(con)
            children.append((set(tuple(x for x in child_state)), action_sub, parent))
    return children

In [11]:
actions = {
    "fly": {
        "action": "(fly ?plane ?to ?from)",
        "conditions": [
            "(plane ?plane)"
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)"
        ],
        "add": ["(at ?plane ?to)"],
        "delete": ["(at ?plane ?from)"]
    },
}
parsed_actions = parse_actions(actions)
start_state = ({('at', '1973', 'SFO'), ('airport', 'SFO'), ('plane', '1973'), ('airport', 'JFK')}, 
                   None, 
                   None)
children = successors(start_state, parsed_actions)
(curr, action, parent) = children[0]
assert ('at', '1973', 'JFK') in curr
assert ('at', '1973', 'SFO') not in curr
assert ('airport', 'SFO') in curr and ('airport', 'JFK') in curr
assert ('plane', '1973') in curr

## is_explored

`is_explored` takes the place of an explored list in `forward_planner` by taking advantage of the nested nature of the states. Each state has a parent as its third parameter, which in turn is a nested tuple of states. By checking whether `child_state` is within the nested tuples of `curr`, the function returns whether or not the child state has been visited. If the child is found as a subset of any parent state, the function returns True - otherwise, it returns False. **Used by**: [forward_planner](#forward_planner).

* **curr**: the state with nested tuples to check for the existence of `child_state`
* **child_state**: the child state being checked

**returns**: `Bool`: whether or not `child_state` has been explored in `curr`.

In [12]:
def is_explored(curr, child_state):
    (curr_state, action, parent) = curr
    while parent != None:
        if child_state <= curr_state:
            return True
        (curr_state, action, parent) = parent
    return child_state <= curr_state

In [13]:
# assertions/unit tests
curr_state = ({('airport', 'SFO'), ('plane', '1973'), ('at', '1973', 'JFK'), ('airport', 'JFK'), ('airport', 'ORD')}, 
              {('fly', '1973', 'JFK', 'SFO')}, 
              (
                  {('airport', 'SFO'), ('plane', '1973'), ('at', '1973', 'SFO'), ('airport', 'JFK'), ('airport', 'ORD')}, 
                  None, 
                  None))
child_state = {('airport', 'SFO'), ('plane', '1973'), ('at', '1973', 'SFO'), ('airport', 'JFK'), ('airport', 'ORD')}
assert is_explored(curr_state, child_state)
child_state = {('airport', 'SFO'), ('plane', '1973'), ('at', '1973', 'ORD'), ('airport', 'JFK'), ('airport', 'ORD')}
assert not is_explored(curr_state, child_state)
child_state = {('airport', 'SFO'), ('plane', '1973'), ('at', '1973', 'JFK'), ('airport', 'JFK'), ('airport', 'ORD')}
assert is_explored(curr_state, child_state)

## get_plan

`get_plan` generates a plan from the current state by taking advantage of the nested tuple nature of a state. By sequentially adding the current state and action to a list, and then unwrapping the parent node, the function can build a list of states and actions using only the `curr` node, which will start with a state list equivalent to the goal of `forward_planner`. The function takes a `debug` parameter that determines whether the plan is of the form: `[S1, A1, S2, A2, S3, A3]` or `[A1, A2, A3]`. If `debug` is true, the first form is chosen; otherwise, the second form is chosen. **Used by**: [forward_planner](#forward_planner).

* **curr**: the current state to unravel to create the plan
* **debug**: a parameter determining the format of `plan`

**returns** `List`: a list representing a plan from a start state to a goal state.

In [14]:
def get_plan(curr, debug):
    (curr_state, action, parent) = curr
    plan = []
    while parent:
        if debug: plan.append(curr_state[0] if type(curr_state) == tuple else curr_state)
        plan.append(action)
        curr_state = parent; 
        action, parent = curr_state[1], curr_state[2]
    if debug: plan.append(curr_state[0])
    return plan[::-1]

In [15]:
# assertions/unit tests
curr_state = ({('airport', 'JFK'), ('at', '1973', 'JFK'), ('airport', 'SFO'), ('plane', '1973')}, 
              {('fly', '1973', 'JFK', 'SFO')}, 
              ({('at', '1973', 'SFO'), ('airport', 'SFO'), ('plane', '1973'), ('airport', 'JFK')}, 
               None, None))
plan = get_plan(curr_state, False)
assert plan == [{('fly', '1973', 'JFK', 'SFO')}]

plan = get_plan(curr_state, True)
actual = [{('at', '1973', 'SFO'), ('plane', '1973'), ('airport', 'SFO'), ('airport', 'JFK')}, 
          {('fly', '1973', 'JFK', 'SFO')}, 
          {('at', '1973', 'JFK'), ('airport', 'SFO'), ('plane', '1973'), ('airport', 'JFK')}]
assert plan == actual

curr_state = ({('plane', '1973'), ('at', '1973', 'JFK'), ('airport', 'SFO'), ('airport', 'ORD'), ('airport', 'JFK')}, 
              {('fly', '1973', 'JFK', 'ORD')}, 
              (
                  {('plane', '1973'), ('at', '1973', 'ORD'), ('airport', 'SFO'), ('airport', 'ORD'), ('airport', 'JFK')}, 
                  {('fly', '1973', 'ORD', 'SFO')}, 
                  (
                      {('plane', '1973'), ('at', '1973', 'SFO'), ('airport', 'SFO'), ('airport', 'ORD'), ('airport', 'JFK')}, 
                      None, None)))
plan = get_plan(curr_state, False)
assert plan == [{('fly', '1973', 'ORD', 'SFO')}, {('fly', '1973', 'JFK', 'ORD')}]

## forward_planner

`forward_planner` implements a forward planner - a type of planning algorithm that takes a list of facts and builds a plan forwards by applying actions, creating new states, and continuing until a goal state is reached. This planner returns a list of total-order plans to go from the start state to the goal state. The function takes a start state, a goal, and a dictionary of all possible actions and returns a list of all possible plans to go from the start to the goal. Additionally, it takes a debug parameter that changes the format of the plans (see `get_plan` for more info).

The algorithm implemented here is two state space searches nested in each other. The states in this algorithm are tuples of the form `(state list, action taken, parent)`. The parent is a nested tuple of states - each state contains a nested tuple of all states taken to get to the current state. This nested tuple arrangement removes the need for an explored list. **Uses**: [parse_state](#parse_state), [parse_actions](#parse_actions), [get_plan](#get_plan), [successors](#successors).

* **start_state**: the start of the plan
* **goal**: the final state in the plan
* **actions**: the dictionary of possible actions
* **debug**: an optional parameter to add states to the generated plans

**returns**: `List[List]`: a list of all possible plans from `start_state` to `goal`.

In [16]:
def forward_planner( start_state, goal, actions, debug=False):
    p_start, p_goal, p_actions = parse_state(start_state), parse_state(goal), parse_actions(actions)
    frontier, plans = [(p_start, None, None)], []
    while frontier:
        (curr_state, action, parent) = frontier.pop()
        if p_goal <= curr_state:
            plan = get_plan((curr_state, action, parent), debug)
            if plan not in plans: plans.append(plan)
            continue
        children = successors((curr_state, action, parent), p_actions)
        for (child, action, parent) in children:
            if not is_explored((curr_state, action, parent), child) or p_goal <= child:
                frontier.append((child, action, parent))
    return plans

In [17]:
actions = {
    "fly": {
        "action": "(fly ?plane ?to ?from)",
        "conditions": [
            "(plane ?plane)"
            "(airport ?to)",
            "(airport ?from)",
            "(at ?plane ?from)"
        ],
        "add": ["(at ?plane ?to)"],
        "delete": ["(at ?plane ?from)"]
    },
}
start_state = ["(plane 1973)", "(airport SFO)", "(airport JFK)", "(at 1973 SFO)"]
goal_state = ["(plane 1973)", "(airport SFO)", "(airport JFK)", "(at 1973 JFK)"]
start_copy = deepcopy(start_state)
plans = forward_planner(start_state, goal_state, actions, False)
assert plans[0] == [('fly', '1973', 'JFK', 'SFO')]
assert start_state == start_copy

start_state = ["(plane 1973)", "(airport SFO)", "(airport ORD)", "(airport JFK)", "(at 1973 SFO)"]
goal_state = ["(plane 1973)", "(airport SFO)", "(airport JFK)", "(at 1973 JFK)"]
plans = forward_planner(start_state, goal_state, actions, False)
assert [('fly', '1973', 'ORD', 'SFO'), ('fly', '1973', 'JFK', 'ORD')] in plans
assert [('fly', '1973', 'JFK', 'SFO')] in plans

You will be solving the problem from above. Here is the start state:

In [18]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [19]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [20]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [21]:
plan = forward_planner( start_state, goal, actions)

In [22]:
for el in plan:
    print(el)

[('drive', 'Me', 'Home', 'Bank'), ('drive', 'Me', 'Bank', 'Store'), ('buy', 'Me', 'Store', 'Drill'), ('drive', 'Me', 'Store', 'Bank'), ('drive', 'Me', 'Bank', 'Home')]
[('drive', 'Me', 'Home', 'Bank'), ('drive', 'Me', 'Bank', 'Store'), ('buy', 'Me', 'Store', 'Drill'), ('drive', 'Me', 'Store', 'Home')]
[('drive', 'Me', 'Home', 'Store'), ('buy', 'Me', 'Store', 'Drill'), ('drive', 'Me', 'Store', 'Bank'), ('drive', 'Me', 'Bank', 'Home')]
[('drive', 'Me', 'Home', 'Store'), ('buy', 'Me', 'Store', 'Drill'), ('drive', 'Me', 'Store', 'Home')]


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.